**Function 2**

Optimising Noisy Models. This corresponds to trying to optimise an unknown machine learning model. However, the initialization of the model is very important, meaning your observations will be very noisy, and the problem might **have a lot of local optima! You are trying to make the model’s log-likelihood as large as possible.**

In [1]:
import torch 
import numpy as np
import matplotlib.pyplot as plt
import plotly
import os
from sklearn.gaussian_process import GaussianProcessRegressor


from botorch.models import SingleTaskGP
from botorch  import fit_gpytorch_model

from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf


In [2]:
f2_inputs = np.load("initial_inputs1.npy")
f2_outputs = np.load("initial_outputs1.npy")
f2_inputs_2 = np.load("initial_inputs2.npy")
f2_outputs_2 = np.load("initial_outputs2.npy")

In [3]:
f2_inputs = np.concatenate((f2_inputs, f2_inputs_2)) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.773955, 0.818842]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.516179, 0.852595]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.682636, 0.906564]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.657373, 0.92707 ]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.738282, 0.94505 ]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.695969, 0.942119]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.695969, 0.942119]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.722636, 0.906564]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.650546, 0.929693]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.742636, 0.904341]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.702733, 0.458018]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.717983, 0.664498]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.704501, 0.451105]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.705161, 0.457239]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.702733, 0.418018]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.742733, 0.435796]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.702733, 0.458018]]))) 
f2_inputs = np.concatenate((f2_inputs, np.array([[0.720511, 0.458018]]))) 

f2_outputs = np.concatenate((f2_outputs,f2_outputs_2)) 
f2_outputs = np.append(f2_outputs,0.26215841751744595)
f2_outputs = np.append(f2_outputs,0.30798911881434576)
f2_outputs = np.append(f2_outputs,0.5004746350462992)
f2_outputs = np.append(f2_outputs,0.4295013179054613)
f2_outputs = np.append(f2_outputs,0.41771047535557376)
f2_outputs = np.append(f2_outputs,0.4720670123276774)
f2_outputs = np.append(f2_outputs,0.5825927664580692)
f2_outputs = np.append(f2_outputs,0.6659374421056163)
f2_outputs = np.append(f2_outputs,0.33114062352865725)
f2_outputs = np.append(f2_outputs,0.43942137201542275)
f2_outputs = np.append(f2_outputs,0.6400642111494899)
f2_outputs = np.append(f2_outputs,0.2962440276848424)
f2_outputs = np.append(f2_outputs,0.6921593595462521)
f2_outputs = np.append(f2_outputs,0.6363815236340811)
f2_outputs = np.append(f2_outputs,0.481611594015654)
f2_outputs = np.append(f2_outputs,0.4107324217400383)
f2_outputs = np.append(f2_outputs,0.6640016362646336)
f2_outputs = np.append(f2_outputs,0.5446540910696784)


In [4]:
variable_value = np.linspace(1, 10, 10)
n_dims = 2 
beta =0.001# 0.039696969696969696
np.random.seed(42)
train_X = torch.tensor(f2_inputs)
train_Y = torch.tensor(f2_outputs).reshape(len(f2_outputs),-1)
best_y = np.max(train_Y.numpy())
best_x = train_X[np.argmax(train_Y.numpy())]
noise_level =0.01# 10
candidate_list = []
best_x_list = []

#train_X = train_X1[0].reshape(8,-1)
#train_Y = train_Y1[0].reshape(1,-1)

for i in range(len(variable_value)):
    noise_level = variable_value[i]
   

    def function_2(x1,x2, noise_level=noise_level):
        result = np.sin(5 * x1+4*x2) * (1 - np.tanh(x1 ** 2) - np.tanh(x2))\
        + np.random.randn() * noise_level
        return result
    
    
    gp = SingleTaskGP(train_X, train_Y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)

    UCB = UpperConfidenceBound(gp, beta=beta)

    bounds = torch.stack([torch.zeros(n_dims), torch.ones(n_dims)])
    candidate, _  = optimize_acqf(
        UCB, bounds=bounds, q=1, num_restarts=50, raw_samples=500,
    )


    x1,x2 = candidate.numpy().ravel().tolist()
    # Get the next value for the function from the script

    y_next = function_2(x1,x2)

    if y_next > best_y:
        best_y = y_next
        best_x = x1,x2


    y_next = torch.tensor(y_next).unsqueeze(-1)

    train_X = torch.cat([train_X,candidate])
    train_Y = torch.cat([train_Y,y_next.unsqueeze(-1)])
    candidate_list.append(candidate) 
    best_x_list.append(best_x)
    #print(f"Next point: {candidate} \n Next Y {y_next}  Best_y, {best_y}")
    #print(f"beta :{beta} noise_level :{noise_level}")
    #print(f"Next point: {candidate} \n Next Y {y_next}")
    #print(f"best_x:{best_x}Best_y,{best_y}")
    

In [5]:
best_x_list

[tensor([0.7227, 0.4380], dtype=torch.float64),
 tensor([0.7227, 0.4380], dtype=torch.float64),
 (0.7113729119300842, 0.4506097137928009),
 (0.7019621729850769, 0.45826372504234314),
 (0.7019621729850769, 0.45826372504234314),
 (0.7019621729850769, 0.45826372504234314),
 (0.7133287787437439, 0.4664672911167145),
 (0.7133287787437439, 0.4664672911167145),
 (0.7133287787437439, 0.4664672911167145),
 (0.7133287787437439, 0.4664672911167145)]